In [1]:
import logging
import sys
import nest_asyncio

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

In [2]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.evaluation import CorrectnessEvaluator
import pandas as pd

pd.set_option("display.max_colwidth", 0)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")
pinecone_index = pinecone.Index("tailwind-ada")

2023-11-25 13:29:06.772 
  command:

    streamlit run /Users/federicoiglesias/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]


AttributeError: st.secrets has no attribute "openai_key". Did you forget to add it to secrets.toml or the app settings on Streamlit Cloud? More info: https://docs.streamlit.io/streamlit-cloud/get-started/deploy-an-app/connect-to-data-sources/secrets-management

In [ ]:
pinecone_index = pinecone.Index("tailwind-ada")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [ ]:
evaluator = CorrectnessEvaluator(service_context=service_context)

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)
query = "What is border-radius? And how do I use it in TailwindCSS?"
response = chat_engine.chat(query)
reference = "It is a property that allows you to round the corners of an element. You can use it in TailwindCSS by adding the rounded class to an element. For example, rounded-lg will round the corners of an element by 0.5rem."

In [ ]:
result = evaluator.evaluate(
    query=query,
    response=response.response,
    reference=reference,
)

print("response: ", response.response)
print("result.score: ", result.score)
print("result.feedback: ", result.feedback)

,Response,Source,Evaluation Result
0,"Border-radius is a CSS property that is used to round the corners of an element. In TailwindCSS, you can use the `border-radius` utility classes to easily apply different border radius sizes to your elements. By default, TailwindCSS provides five border radius size utilities: `rounded-none`, `rounded-sm`, `rounded`, `rounded-md`, `rounded-lg`, and `rounded-full`. These classes can be applied to any element to round its corners with the corresponding radius size. If you want to customize the default border radius sizes in TailwindCSS, you can edit the `theme.borderRadius` section of your Tailwind config file. You can change, add, or remove the border radius sizes according to your needs. For example, if you want to change the default border radius size to `4px` and add a new size called `large` with a radius of `12px`, you can modify the `theme.borderRadius` section in your Tailwind config file as follows: ```diff-js module.exports = { theme: { borderRadius: { 'none': '0', 'DEFAULT': '4px', 'large': '12px', } } } ``` After customizing the border radius sizes, you can use the updated utility classes in your HTML markup to apply the desired border radius to your elements. Additionally, if you need to apply arbitrary border radius values that are not predefined in TailwindCSS, you can use the `rounded-[value]` class. Replace `[value]` with the desired pixel value or any other valid CSS unit. For example, if you want to apply a border radius of `12px` to an element, you can use the `rounded-[12px]` class. Remember to include the TailwindCSS stylesheet in your project to make use of the border radius utility classes.","--- ## Applying conditionally ### Hover, focus, and other states ### Breakpoints and media queries --- ## Using custom values ### Customizing your theme By default, Tailwind provides five border radius size utilities. You can change, add, or remove these by editing the `theme.borderRadius` section of your Tailwind config. ```diff-js {{ filename: 'tailwind.config.js' }} module.exports = { theme: { borderRadius: { 'none': '0', - 'sm': '0.125rem', - DEFAULT: '0.25rem', + DEFAULT: '4px', - 'md': '0.375rem', - 'lg': '0.5rem', - 'full': '9999px', + 'large': '12px', } } } ``` Learn more about customizing the default theme in the [theme customization](/docs/theme#customizing-t...",Pass
